In [139]:
import json
import os
import cv2 
import numpy as np
from PIL import Image, ImageDraw
import random

In [140]:
def generate_color_map(categories):
    """Generate a fixed color map for categories"""
    random.seed(42)  # For reproducible colors
    return {cat: (random.randint(0, 255), 
                 random.randint(0, 255),
                 random.randint(0, 255)) for cat in categories}

In [141]:
# def draw_cityscapes_annotations(image_path, json_path, color_map):
#     """Draw annotations directly using pixel values."""
#     # Load the image to get its dimensions
#     img = cv2.imread(image_path)
#     height, width = img.shape[:2]

#     # Create a blank 2D array for annotations
#     annotation_map = np.zeros((3, height, width), dtype=np.uint8)
    
#     # Load annotations
#     with open(json_path, 'r') as f:
#         ann_data = json.load(f)
    
#     # Draw each object
#     for obj in ann_data['objects']:
#         label = obj['label']
#         if label in color_map:
#             value = color_map[label]  # Use the integer value directly

#             print(value)
#             # Convert polygon points to tuple list
#             polygon = [(p[0], p[1]) for p in obj['polygon']]
#             if len(polygon) > 2:  # Only draw valid polygons
#                 img = Image.fromarray(annotation_map)  # Convert to PIL for drawing
#                 draw = ImageDraw.Draw(img)
#                 draw.polygon(polygon, fill=value)
#                 annotation_map = np.array(img)  # Convert back to NumPy array
    
#     return annotation_map

# def draw_cityscapes_annotations(image_path, json_path, color_map):
#     """Draw annotations from Cityscapes format JSON"""
#     # Load image
#     img = cv2.imread(image_path)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     print(img.shape)
#     # img = np.ones(img.shape) * 27
#     pil_img = Image.fromarray(img)
#     draw = ImageDraw.Draw(pil_img, 'RGB')
    
#     # Load annotations
#     with open(json_path, 'r') as f:
#         ann_data = json.load(f)
    
#     # Draw each object
#     for obj in ann_data['objects']:
#         label = obj['label']
#         if label in color_map:
#             color = color_map[label]
#             # Add alpha for semi-transparency
#             fill_color = color
            
#             # Convert polygon points to tuple list
#             polygon = [(p[0], p[1]) for p in obj['polygon']]
#             if len(polygon) > 2:  # Only draw valid polygons
#                 draw.polygon(polygon, fill=fill_color)
    
#     return np.array(pil_img)

def draw_cityscapes_annotations(image_path, json_path, color_map):
    """Draw annotations from Cityscapes format JSON"""
    # Load the original image to get its shape
    img = cv2.imread(image_path)
    height, width = img.shape[:2]
    
    # Create a blank image with the same dimensions and fill it with (27, 27, 27)
    blank_image = np.full((height, width, 3), (27, 27, 27), dtype=np.uint8)
    
    # Convert to PIL Image for drawing
    pil_img = Image.fromarray(blank_image)
    draw = ImageDraw.Draw(pil_img, 'RGB')
    
    # Load annotations
    with open(json_path, 'r') as f:
        ann_data = json.load(f)
    
    # Draw each object
    for obj in ann_data['objects']:
        label = obj['label']
        if label in color_map:
            color = color_map[label]
            
            # Convert polygon points to tuple list
            polygon = [(p[0], p[1]) for p in obj['polygon']]
            if len(polygon) > 2:  # Only draw valid polygons
                draw.polygon(polygon, fill=color)
    
    # Convert back to NumPy array and return
    return np.array(pil_img)


In [142]:
categories = {'road': 1,
 'parking': 2,
 'drivable fallback': 2,
 'sidewalk': 3,
 'non-drivable fallback': 4,
 'rail track': 4,
 'person': 5,
 'animal': 5,
 'rider': 6,
 'motorcycle': 7,
 'bicycle': 8,
 'autorickshaw': 9,
 'car': 10,
 'truck': 11,
 'bus': 12,
 'caravan': 13,
 'trailer': 13,
 'vehicle fallback': 13,
 'curb': 14,
 'wall': 15,
 'fence': 16,
 'guard rail': 17,
 'billboard': 18,
 'traffic sign': 19,
 'traffic light': 20,
 'pole': 21,
 'pole group': 21,
 'obs-str-bar-fallback': 22,
 'building': 23,
 'bridge': 24,
 'tunnel': 24,
 'vegetation': 25,
 'sky': 26,
 'fallback background': 26,
 'unlabeled': 27,
 'out of ROI': 27}

categories = {
    key : (categories[key], categories[key], categories[key]) for key in categories
}

In [143]:
shapes = {}

In [144]:
def convert_image(image_path, output_path, color_map) :
    
    # Get corresponding annotation paths
    cityscapes_json = image_path.replace('leftImg8bit', 'gtFine').replace('_gtFine.png', '_gtFine_polygons.json')    
    
    # Generate annotation map
    annotation_map = draw_cityscapes_annotations(image_path, cityscapes_json, color_map)

    # annotation_map = np.array([
    #     annotation_map.copy(),
    #     annotation_map.copy(),
    #     annotation_map.copy()
    # ])

    shape = annotation_map.shape


    # if str(shape) not in shapes : 
    #     shapes[str(shape)] = 1
    # else :
    #     shapes[str(shape)] += 1


    cv2.imwrite(output_path, cv2.cvtColor(annotation_map, cv2.COLOR_RGB2BGR))
    #     print(f"Saved comparison to {output_path}")

    # # Convert the NumPy array to a PIL Image
    # image = Image.fromarray(annotation_map, mode='RGB')

    # # Save the image
    # image.save(output_path)

In [145]:
def main():
    # Set paths
    # data_dir = "/home/gaurav/Documents/Semester 5/DL/DL Project/Custom Mode/IDD_Segmentation_Flattened/images" # dataset path
    data_dir = r"/home/gaurav/Documents/Semester 5/DL/DL Project/Final Output Conversion/temp_dataset/images" # dataset path
    
    # out_base_dir = "/home/gaurav/Documents/Semester 5/DL/DL Project/Custom Mode/IDD_Segmentation_Flattened/labels" # output path
    out_base_dir = r"/home/gaurav/Documents/Semester 5/DL/DL Project/Final Output Conversion/temp_dataset/labels" # output path
    
    folders = ['train', 'val']
    # folders = ['val']

    color_map = categories

    ctr = 0

    for folder in folders : 
        dir_path = rf"{data_dir}/{folder}"
        out_path = rf"{out_base_dir}/{folder}"

        image_files = []
        for root, _, files in os.walk(dir_path):
            for f in files:
                if f.endswith('_leftImg8bit.png'):


                    image_path = f"{dir_path}/{f}"
                    output_path = f"{out_path}/{f}"

                    # print(f)

                    convert_image(image_path, output_path, color_map)

                    ctr += 1
                
                if ctr % 100 == 0 : print(f"{ctr} images converted!")

    for folder in ['train', 'val'] :
        files = os.listdir(f'./temp_dataset/labels/{folder}/')

        with open(f'./temp_dataset/{folder}.txt', 'a+') as f_obj :
            for file in files : 
                f_obj.write(f'./temp_dataset/images/{folder}/{file} ./temp_dataset/labels/{folder}/{file}\n')

        

if __name__ == "__main__":
    main()
    # print(shapes)

0 images converted!
0 images converted!


In [146]:
# import json
# import os
# import cv2 
# import numpy as np
# from PIL import Image, ImageDraw
# import random

# def generate_color_map(categories):
#     """Generate a fixed color map for categories"""
#     random.seed(42)  # For reproducible colors
#     return {cat: (random.randint(0, 255), 
#                  random.randint(0, 255),
#                  random.randint(0, 255)) for cat in categories}

# def draw_cityscapes_annotations(image_path, json_path, color_map):
#     """Draw annotations from Cityscapes format JSON"""
#     # Load image
#     img = cv2.imread(image_path)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     pil_img = Image.fromarray(img)
#     draw = ImageDraw.Draw(pil_img, 'RGBA')
    
#     # Load annotations
#     with open(json_path, 'r') as f:
#         ann_data = json.load(f)
    
#     # Draw each object
#     for obj in ann_data['objects']:
#         label = obj['label']
#         if label in color_map:
#             color = color_map[label]
#             # Add alpha for semi-transparency
#             fill_color = color + (127,)
#             outline_color = color + (255,)
            
#             # Convert polygon points to tuple list
#             polygon = [(p[0], p[1]) for p in obj['polygon']]
#             if len(polygon) > 2:  # Only draw valid polygons
#                 draw.polygon(polygon, fill=fill_color, outline=outline_color)
    
#     return np.array(pil_img)

# def main():
#     # Set paths
#     data_dir = "DatasetIDD"  # Adjust this to your dataset path
#     train_dir = os.path.join(data_dir, "leftImg8bit/train")
    
#     # Categories
#     categories = [
#         'road', 'parking', 'drivable fallback',
#         'sidewalk', 'non-drivable fallback', 'rail track',
#         'person', 'animal', 'rider',
#         'motorcycle', 'bicycle', 'auto-rickshaw', 'car', 'truck', 'bus', 
#         'caravan', 'trailer', 'vehicle fallback',
#         'curb', 'wall', 'fence', 'guard rail', 'billboard', 'traffic sign', 
#         'traffic light', 'pole', 'pole group', 'obstacle/bar fallback',
#         'building', 'bridge', 'tunnel',
#         'sky',
#         'fallback background', 'unlabeled', 'out of ROI'
#     ]
    
#     # Generate color map
#     color_map = generate_color_map(categories)
    
#     # Find all image files
#     # image_files = []
#     # for root, _, files in os.walk(train_dir):
#     #     for f in files:
#     #         if f.endswith('_leftImg8bit.png'):
#     #             image_files.append(os.path.join(root, f))
    
#     # if not image_files:
#     #     print("No images found!")
#     #     return
    
#     # Select a random image
#     # image_path = random.choice(image_files)
#     image_path = "DatasetIDD/leftImg8bit/train/000128_leftImg8bit.png"
#     image_name = os.path.basename(image_path)
#     print(f"Selected image: {image_name}")
    
#     # Get corresponding annotation paths
#     cityscapes_json = image_path.replace('leftImg8bit', 'gtFine').replace('_gtFine.png', '_gtFine_polygons.json')
#     # cityscapes_json = image_path.replace('leftImg8bit.png', 'gtFine_polygons.json')
#     coco_json = os.path.join(data_dir, 'annotations/instancesonly_filtered_gtFine_train.json')
    
#     # Draw annotations
#     cityscapes_viz = draw_cityscapes_annotations(image_path, cityscapes_json, color_map)
#     coco_viz = draw_coco_annotations(image_path, coco_json, color_map, image_name)
    
#     # Combine images side by side
#     combined = np.hstack((cityscapes_viz, coco_viz))
    
#     # Save result
#     output_path = 'annotation_comparison.png'
#     cv2.imwrite(output_path, cv2.cvtColor(combined, cv2.COLOR_RGB2BGR))
#     print(f"Saved comparison to {output_path}")
    
#     # Print legend
#     print("\nColor Legend:")
#     for cat, color in color_map.items():
#         print(f"{cat}: RGB{color}")

# if __name__ == "__main__":
#     main()